<a href="https://colab.research.google.com/github/Ahtesham519/jupyter-notebook_python/blob/main/nlp_code_ch8_8_livedemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install scikit-learn
!pip lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: unknown command "lime"


In [3]:
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

#import feature extraction methods from sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import _stop_words
from sklearn.metrics import accuracy_score

#pre-processing of text
import string
import re

#import classifiers from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [6]:
#loading the datasset
try:
  from google.colab import files
  uploaded = files.upload()
  our_data = pd.read_csv("Full-Economic-News-DFE-839861.csv" , encoding = "ISO-8859-1")
except ModuleNotFoundError:
  our_data = pd.read_csv(os.getcwd() + "\Data\Full-Economic-News-DFE-839861.csv" , encoding = "iso-88859-1")



Saving Full-Economic-News-DFE-839861.csv to Full-Economic-News-DFE-839861.csv


In [9]:
our_data["relevance"].value_counts()/our_data.shape[0] #Class distribution in the dataset
#Convert label to a numerical variable
our_data = our_data[our_data.relevance != "not sure"]
our_data['relevance'] = our_data.relevance.map({'yes' :1 , 'no':0})  #relevant is 1 , not-relevant is 0
our_data = our_data[["text" , "relevance"]]   #Let us take only the two columns we need.

our_data.shape

(7991, 2)

In [10]:
#train-test split
X = our_data.text #the column text contains textual data to extract features from 
y = our_data.relevance #this is the column we are learning to predict.
print(X.shape , y.shape)

#split X and y into training and testing sets . By Default , it splits 75% training and 25% test
#random_state - 1 and reproducibility
X_train , X_test , y_train , y_test = train_test_split(X, y, random_state = 1)
print(X_train.shape , y_train.shape)
print(X_test.shape , y_test.shape)

(7991,) (7991,)
(5993,) (5993,)
(1998,) (1998,)


In [29]:
our_data

,text,relevance
0,NEW YORK -- Yields on most certificates of dep...,1
1,The Wall Street Journal Online</br></br>The Mo...,0
2,WASHINGTON -- In an effort to achieve banking ...,0
3,The statistics on the enormous costs of employ...,0
4,NEW YORK -- Indecision marked the dollar's ton...,1
...,...,...
7995,Secretary of Commerce Charles W. Sawyer said y...,1
7996,"U.S. stocks inched up last week, overcoming co...",0
7997,Ben S. Bernanke cleared a key hurdle Thursday ...,0
7998,The White House's push to contract out many fe...,0


In [34]:
#Pre-processing and vectorinzing
stopswords = _stop_words.ENGLISH_STOP_WORDS
def clean(doc) :    #doc is a string of text
  doc = doc.replace("</br>" , " ")   #This text contains a lot of <br/> tags.
  doc = " ".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
  doc = " ".join([token for token in doc.split() if token not in stopswords])

  #remove punctuation and numbers
  return doc



In [36]:
our_data

,text,relevance
0,NEW YORK -- Yields on most certificates of dep...,1
1,The Wall Street Journal Online</br></br>The Mo...,0
2,WASHINGTON -- In an effort to achieve banking ...,0
3,The statistics on the enormous costs of employ...,0
4,NEW YORK -- Indecision marked the dollar's ton...,1
...,...,...
7995,Secretary of Commerce Charles W. Sawyer said y...,1
7996,"U.S. stocks inched up last week, overcoming co...",0
7997,Ben S. Bernanke cleared a key hurdle Thursday ...,0
7998,The White House's push to contract out many fe...,0


In [33]:
#preprocess and Vectorize train and test data 
vect = CountVectorizer(preprocessor = clean , stop_words='english')   #instantiate a vectoriezer
X_train_dtm = vect.fit_transform(X_train , y= None)   #use it to extract features from training data.


#transform testing data(using training data's features )
X_test_dtm = vect.transform(X_test)
print(X_train_dtm.shape, X_test_dtm.shape)        
#i.e , the dimension of our feature vector is 49753

TypeError: ignored

In [37]:
#Train a classifiers
vect =CountVectorizer(preprocessor = clean , max_features = 1000)   #step-1
X_train_dtm = vect.fit_transform(X_train)  #combined step2 and step3
X_test_dtm = vect.transform(X_test)

classifier = LogisticRegression(class_weight = 'balanced')   #instantiate a logistic regression model
classifier.fit(X_train_dtm, y_train)   #fit the model with training data

#make predictions on test data
y_pred_class = classifier.predict(X_test_dtm)

#Claculate evaluation measure 
print("accuracy: " , accuracy_score(y_test , y_pred_class))


#until here , it is the same code as earlier

ValueError: ignored

Part -2 Usnig lime to intercept predictions 

In [38]:
!pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283859 sha256=f29d86343704c1a740db5e73f7d2b632ccd2fcc5449ecccc736092ab9081a13e
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [40]:
#part -2 : Using lime to intercept predictions

from lime import lime_text
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline

y_pred_prob = classifier.predict_proba(X_test_dtm) [: , 1]
c = make_pipeline(vect , classifier)
mystring = list(X_test )[221]  #Take a string from test instance
print(c.predict_proba([mystring]))   #Prediction is a "NO" here . i.e , not releavent
class_names = ["no" , "yes"]   #not relevant , relavant
explainer = LimeTextExplainer(class_names = class_names)
exp = explainer.explain_instance(mystring , c.predict_proba , num_features = 6)

exp.as_list()

NameError: ignored

In [41]:
%matplotlib inline
fig = exp.as_pyplot_figure()

NameError: ignored